In [23]:
#HW implementation

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

t = [0.25, 0.5, 1, 2, 3, 5, 7, 10, 20]
zero_curve = [0.008, 0.0085, 0.0092, 0.01,  0.0106, 0.0119, 0.0132, 0.015, 0.02]

sig = 0.01
a = 0.1
fixed_rate = 0.0149
Notional = 1000

timesteps = np.arange(0, 10.25, 0.25)

#print(len(timesteps))

cs = CubicSpline(t, zero_curve)
spot_rate = cs(timesteps)
df = np.exp(- timesteps * spot_rate)

#print(df)

dt = np.zeros(len(timesteps))
fwd = np.zeros(len(timesteps))
beta = np.zeros(len(timesteps))
B = np.zeros(len(timesteps))
fwdswapval = np.zeros(len(timesteps))

fwd[0] = spot_rate[0]
for i in range(1, len(timesteps)):
    dt[i] = timesteps[i] - timesteps[i - 1]
    fwd[i] = -np.log(df[i]/df[i - 1])/dt[i]
    beta[i] = fwd[i] + (sig * (1 - np.exp(-a *  timesteps[i]))/ (a * np.sqrt(2)))**2
    B[i] = (1 - np.exp(-a * timesteps[i])) / a
    

for i in range(1, len(timesteps) ):
    fwdswapval[i -1] = np.sum(df[i:len(timesteps)] * dt[i - 1] * fixed_rate) - (df[i - 1] - df[len(timesteps) - 1])
    
df_calib = np.zeros((len(timesteps), len(timesteps)))

for i in range(len(timesteps)):
    for j in range(i, len(timesteps)):
        df_calib[i, j] = df[j] / df[i] * np.exp(B[j - i] * fwd[i] - (sig ** 2) * (1 - np.exp(-2 * a * timesteps[i])) * (B[j - i] ** 2) / 4 *a)
        
#print(df_calib)

param1 = np.exp(-a * 0.25)  # 0.25 should be dt[i] 
param2 = np.sqrt((1 - np.exp(-2 * a * 0.25))/(2 * a)) #again hard coded dt


short_rate = np.zeros(len(timesteps) - 1)

sims = 1000

df_sim = np.zeros((sims, len(timesteps) - 1, len(timesteps) - 1))

short_rate[0] = spot_rate[0]
for k in range(len(timesteps) - 1):
    df_sim[:, 0, k] = df_calib[0, k] * np.exp(- B[k + 1] * short_rate[0])
    #print(B[k + 1])
    #print(short_rate[0])
    
df_sim[:, 0, 0] = 1

MtM = np.zeros((sims, len(timesteps)))

fwdswapval_sim = np.zeros((sims, len(timesteps) - 1, len(timesteps) - 1)) 

for i in range(sims):
    for j in range(1, len(timesteps) - 1):
        sumfxd = 0
        sumflt = 0
        z1 = np.random.normal(0,1)
        short_rate[j] = short_rate[j - 1] * param1 + z1 * sig * param2 + beta[j] - beta[j - 1] * param1
        for k in range(len(timesteps) - 2):
            df_sim[i,  j,  k] = df_calib[j, k] * np.exp(- B[k + 1 - j] * short_rate[j])
            df_sim[i, j, j] = 1
        
        for k in range(len(timesteps) - 1):
             fwdswapval_sim[i, j -1, k - 1] = np.sum(df_sim[i, j:len(timesteps)] * dt[j - 1] * fixed_rate) - (df[j - 1] - df[len(timesteps) - 1])
        
        for k in range(j,  len(timesteps)  - 2):
            if df_sim[i, j, k] != 0:
                fwdswapval_sim[i, j, k + 1] = - np.log(df_sim[i, j, k + 1] / df_sim[i, j, k]) / 0.25
            else:
                fwdswapval_sim[i, j, k + 1] = 0
            
            sumfxd += 0.25 * fixed_rate * df_sim[i, j, k + 1] * Notional
            sumflt += 0.25 * fwdswapval_sim[i, j, k] * df_sim[i, j, k + 1] * Notional

        MtM[i, j] = sumflt - sumfxd


EPE = np.mean(np.where(MtM > 0, MtM, 0), axis=0)
print(EPE)

ENE = np.mean(np.where(MtM < 0, MtM, 0), axis=0)
print(ENE)

#EPE_ENE = pd.DataFrame({'EPE' : EPE, 'ENE' : ENE})
#EPE_ENE.to_csv('exposure_results.csv', index=False)

#broadly correct - few bugs possibly 



C:\Users\Laptop\AppData\Local\Temp\ipykernel_21024\700325490.py:88: RuntimeWarning: divide by zero encountered in log
  fwdswapval_sim[i, j, k + 1] = - np.log(df_sim[i, j, k + 1] / df_sim[i, j, k]) / 0.25


[ 0.         37.34462458 39.78473526 41.34664937 42.87625647 42.69615339
 43.74555999 44.82946716 45.57958568 46.49442456 46.00662115 46.3080093
 46.80177456 45.85360825 46.65050078 46.90201132 45.63441799 45.18223379
 43.81144961 42.24267626 41.63014871 40.74473508 38.88513602 37.2691801
 35.48376632 34.60901104 32.44710668 30.17779855 27.97683169 25.98001679
 23.37946465 20.9647207  18.64833397 15.80694773 12.8925342   9.8446804
  6.62134427  0.          0.          0.          0.        ]
[  0.          -1.16025013  -3.12629897  -5.00088162  -6.6775107
  -8.52864791  -9.17252519 -10.39082953 -11.144388   -12.07699727
 -12.58919782 -13.36270504 -14.08384963 -13.68719013 -14.03827703
 -13.94035248 -13.55697124 -13.38291488 -13.0044423  -12.59280924
 -12.25212297 -12.19615579 -11.66896899 -11.3028985  -10.88381285
 -10.15017749  -9.95213342  -9.27627639  -8.85095849  -8.04268136
  -7.27489044  -6.79901622  -5.88069272  -4.96599245  -4.01420088
  -3.08431748  -2.07773483  -3.67919467   